# Tabular Model: sequential data, with context

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mostly-ai/mostlyai-engine/blob/main/examples/sequential.ipynb)

In [40]:
from pathlib import Path
import pandas as pd
import numpy as np
from mostlyai import engine

# init workspace and logging
ws = Path("ws-tabular-sequential")
engine.init_logging()

# load original data
url = "https://github.com/mostly-ai/public-demo-data/raw/refs/heads/dev/baseball"
trn_ctx_df = pd.read_csv(f"{url}/players.csv.gz")  # context data
trn_tgt_df = pd.read_csv(f"{url}/batting.csv.gz")[["players_id", "year"]]  # target data
trn_tgt_df["year"] = trn_tgt_df["year"].apply(lambda x: str(x) + "-01-01")
trn_ctx_df = trn_ctx_df[trn_ctx_df["id"] == "aardsda01"]
trn_tgt_df = trn_tgt_df[trn_tgt_df["players_id"] == "aardsda01"]
trn_tgt_df["year"] = pd.Series(["2023-01-01 10:00:00", "2024-01-01 10:00:00", "2024-01-01 10:30:00", np.nan, "2024-01-01 20:00:00", "2024-01-02 20:00:00", "2024-01-02 23:03:03", np.nan, "2024-01-12 23:03:03"])
display(trn_ctx_df)
display(trn_tgt_df)


,id,country,birthDate,deathDate,nameFirst,nameLast,weight,height,bats,throws
0,aardsda01,USA,1981-12-27,NaN,David,Aardsma,215.0,75.0,R,R


,players_id,year
0,aardsda01,2023-01-01 10:00:00
1,aardsda01,2024-01-01 10:00:00
2,aardsda01,2024-01-01 10:30:00
3,aardsda01,NaN
4,aardsda01,2024-01-01 20:00:00
5,aardsda01,2024-01-02 20:00:00
6,aardsda01,2024-01-02 23:03:03
7,aardsda01,NaN
8,aardsda01,2024-01-12 23:03:03


In [41]:
# execute the engine steps
engine.split(                         # split data as PQT files for `trn` + `val` to `{ws}/OriginalData/(tgt|ctx)-data`
  workspace_dir=ws,
  tgt_data=trn_tgt_df,
  ctx_data=trn_ctx_df,
  tgt_context_key="players_id",
  ctx_primary_key="id",
  model_type="TABULAR",
  tgt_encoding_types={"year": "TABULAR_DATETIME_RELATIVE"},
)
engine.analyze(workspace_dir=ws, value_protection=False)      # generate column-level statistics to `{ws}/ModelStore/(tgt|ctx)-data/stats.json`
engine.encode(workspace_dir=ws)       # encode training data to `{ws}/OriginalData/encoded-data`

[2025-10-09 17:37:44,480] INFO   : SPLIT started
[2025-10-09 17:37:44,481] INFO   : clean `ws-tabular-sequential/OriginalData/tgt-data`
[2025-10-09 17:37:44,482] INFO   : clean `ws-tabular-sequential/OriginalData/tgt-meta`
[2025-10-09 17:37:44,483] INFO   : clean `ws-tabular-sequential/OriginalData/ctx-data`
[2025-10-09 17:37:44,483] INFO   : clean `ws-tabular-sequential/OriginalData/ctx-meta`
[2025-10-09 17:37:44,484] INFO   : model_type='TABULAR'
[2025-10-09 17:37:44,484] INFO   : tgt_encoding_types={'year': 'TABULAR_DATETIME_RELATIVE'}
[2025-10-09 17:37:44,484] INFO   : ctx_encoding_types={'country': 'TABULAR_CATEGORICAL', 'birthDate': 'TABULAR_CATEGORICAL', 'deathDate': 'TABULAR_CATEGORICAL', 'nameFirst': 'TABULAR_CATEGORICAL', 'nameLast': 'TABULAR_CATEGORICAL', 'weight': 'TABULAR_NUMERIC_AUTO', 'height': 'TABULAR_NUMERIC_AUTO', 'bats': 'TABULAR_CATEGORICAL', 'throws': 'TABULAR_CATEGORICAL'}
[2025-10-09 17:37:44,490] INFO   : SPLIT finished in 0.01s
[2025-10-09 17:37:44,490] INFO  

In [42]:
encoded_df = pd.read_parquet(ws / "OriginalData" / "encoded-data")
itt_columns = [c for c in encoded_df if c.startswith("tgt:t0/c0")]

In [43]:
encoded_df.iloc[0]

ctxflt:t0/c0__cat                                         1
ctxflt:t1/c1__cat                                         1
ctxflt:t2/c2__cat                                         1
ctxflt:t3/c3__cat                                         1
ctxflt:t4/c4__cat                                         1
ctxflt:t5/c5__cat                                         1
ctxflt:t6/c6__cat                                         1
ctxflt:t7/c7__cat                                         1
ctxflt:t8/c8__cat                                         1
tgt:/__sidx_cat              [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
tgt:/__slen_cat              [9, 9, 9, 9, 9, 9, 9, 9, 9, 9]
tgt:/__ridx_cat              [9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
tgt:t0/c0__nan               [0, 0, 0, 1, 0, 0, 0, 1, 0, 0]
tgt:t0/c0__start_year        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
tgt:t0/c0__start_month       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
tgt:t0/c0__start_day         [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
tgt:t0/c0__start_hour        [0, 0, 0, 0

In [ ]:
engine.train(                         # train model and store to `{ws}/ModelStore/model-data`
    workspace_dir=ws,
    max_training_time=2,              # limit TRAIN to 2 minute for demo purposes
)
engine.generate(workspace_dir=ws)     # use model to generate synthetic samples to `{ws}/SyntheticData`

In [ ]:
# load synthetic data
syn_tgt_df = pd.read_parquet(ws / "SyntheticData")
syn_tgt_df.head(5)

### QUALITY ASSURANCE

#### sequence lengths

In [ ]:
trn_seq_lens = trn_tgt_df.groupby("players_id").size()
syn_seq_lens = syn_tgt_df.groupby("players_id").size()

In [ ]:
print("tgt: ", np.quantile(trn_seq_lens, np.arange(0, 1.1, 0.1), method="inverted_cdf"))
print("syn: ", np.quantile(syn_seq_lens, np.arange(0, 1.1, 0.1), method="inverted_cdf"))

#### coherence

In [ ]:
syn_avg_teams_per_player = syn_tgt_df.groupby("players_id")["team"].nunique().mean().round(1)
trn_avg_teams_per_player = trn_tgt_df.groupby("players_id")["team"].nunique().mean().round(1)
syn_avg_teams_per_player, trn_avg_teams_per_player